**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [31]:
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json
import ipdb
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential,model_from_json
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization

# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

At the start of the game, the agent has not yet explored all the possibilities, therefore all the rewards possible.
If he always goes for the action learnt by his current policy (giving him the maximum short term reward for ex), then he will always do the same thing, and maybe miss an action giving him less reward immediately, but will give him later a way better total reward.
epsilon is there to ensure we have at least some exploration to avoid the situation described above.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size, self.grid_size, 3)) + 128
        b[self.board > 0, 0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:] = 256
        b[-2:, :, :] = 0
        b[:, -2:, :] = 0
        b[:2, :, :] = 0
        b[:, :2, :] = 0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:] = b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        #self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""
        
        # random between min=3 and max= grid_size - 3
        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]

        # Binom(1, temperature), size fois
        bonus = 0.5 * np.random.binomial(1, self.temperature, size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0 * np.random.binomial(1, self.temperature, size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))

        # Pour ne pas avoir les malus aux mêmes positions que les bonus
        malus[bonus>0]=0
        
        self.board = bonus + malus
        self.board[self.x,self.y] = 0

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        #self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [6]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=6 # set small when debugging
epochs_test=6 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

I don't understand position. This code is shitty as hell.

board is an array that will represent the current state of the game, which is : the cheese and poisounous trap left, and the position of the agent.

position is an array of the same size keeping track of where the agent went (with 1), and where he can not go (with -1)

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [7]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        random_action = np.random.randint(0, self.n_action)
        # whether or not the action can be performed is checked by the act function
        return random_action

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [8]:
def test(agent, env, epochs, prefix=''):
    # Number of won games
    score = 0
    score = 0
    loss = 0
    
    for e in range(epochs):
        
        ##### FILL IN HERE
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0
        
        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            ## No reinforcement strategy for a random Agent
            ## loss = agent.reinforce(prev_state, state,  action, reward, game_over)
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [9]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

/usr/bin/ffmpeg -y -f rawvideo -pix_fmt rgb24 -s 272x272 -i - /home/johnny/3A/deeplearning/rl_project/random0.mp4
Win/lose count 8.5/15.0. Average score (-6.5)
/usr/bin/ffmpeg -y -f rawvideo -pix_fmt rgb24 -s 272x272 -i - /home/johnny/3A/deeplearning/rl_project/random1.mp4
Win/lose count 9.5/3.0. Average score (0.0)
/usr/bin/ffmpeg -y -f rawvideo -pix_fmt rgb24 -s 272x272 -i - /home/johnny/3A/deeplearning/rl_project/random2.mp4
Win/lose count 14.0/15.0. Average score (-0.3333333333333333)
/usr/bin/ffmpeg -y -f rawvideo -pix_fmt rgb24 -s 272x272 -i - /home/johnny/3A/deeplearning/rl_project/random3.mp4
Win/lose count 14.0/23.0. Average score (-2.5)
/usr/bin/ffmpeg -y -f rawvideo -pix_fmt rgb24 -s 272x272 -i - /home/johnny/3A/deeplearning/rl_project/random4.mp4
Win/lose count 13.0/14.0. Average score (-2.2)
/usr/bin/ffmpeg -y -f rawvideo -pix_fmt rgb24 -s 272x272 -i - /home/johnny/3A/deeplearning/rl_project/random5.mp4
Win/lose count 7.5/8.0. Average score (-1.9166666666666667)
Final scor

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




====================================================================================================================================================================================================================================

First, let's start off from the relation : 
    $$ Q^{\pi}(s, a) = E^{\pi}( G_{t} | S_t = s, A_t = a) $$

Using the cumulative reward $G_t$ property:
    $$ Q^{\pi}(s, a) = E^{\pi}( R_{t+1} + \gamma * G_{t + 1} | S_t = s, A_t = a) $$

$R_{t+1}$ is the reward at step t+1 (after state $S_t$ and action $A_t$)

Separating the expectation:
    $$ Q^{\pi}(s, a) = E^{\pi}( R_{t+1} | S_t = s, A_t = a) + E^{\pi}(\gamma * G_{t + 1} | S_t = s, A_t = a) $$

By definition : $r(s,a) = (R_{t+1} | S_t = s, A_t = a)$
So the first term is : $E^{\pi}( R_{t+1} | S_t = s, A_t = a) = E^{\pi}(r(s,a))$

As for the second term :
$G_{t + 1}$ is independant of $(S_t = s, A_t = a)$. Because $G_{t + 1}$ is the cumulative reward at step t+1, and therefore does not care about the state $S_t$, only the current one $S_{t+1}$. So we have:
    $$E^{\pi}(\gamma * G_{t + 1} | S_t = s, A_t = a) = E^{\pi}(\gamma * G_{t + 1})$$ 

Then using the property the expectation:
    $$E^{\pi}(\gamma * G_{t + 1}) = E^{\pi}(E^{\pi}(\gamma * G_{t + 1} | S_{t+1} = s', A_{t+1} = a')) $$

And the expression that we've made appear is:
    $$E^{\pi}(\gamma * G_{t + 1} | S_{t+1} = s', A_{t+1} = a') = Q^{\pi}(s', a')$$
So 
    $$ E^{\pi}(E^{\pi}(\gamma * G_{t + 1} | S_{t+1} = s', A_{t+1} = a')) =  E^{\pi}(Q^{\pi}(s', a'))$$
This is our second term.

Then if we replace everything:
    $$ Q^{\pi}(s, a) = E^{\pi}(r(s,a)) + E^{\pi}(Q^{\pi}(s', a')) $$
Since $(s, a)$ are fixed, the expectation is acting on $(s', a')$


====================================================================================================================================================================================================================================

If we assume the existence of an optimal policy giving us the optimal action-value function:
    $$ Q^*(s,a) = max_{\pi} Q^{\pi}(s,a) $$

Then it is sure that the policy is always a greedy one, that is always taking the action giving the best reward.
    $$Q^*(s, a) = E_{s' ~ \pi^*(. |s, a)} (r(s,a) + \gamma max_{a'}Q^*(s',a'))$$
We have this expression because we are always taking the best reward.

====================================================================================================================================================================================================================================

The real $Q^*$ is not available to us, so the idea is to approach it dynamically by updating its values step by step using a neural network. If we take the loss:
    $$ \mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.$$
    
And derivate it wrt $\theta$:
    $$ \frac{\partial \mathcal{L}}{\partial \theta} = E_{s' \sim \pi^*(.|s,a)}(2(r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta))*(\gamma \frac{\partial \max_{a'}Q(s',a',\theta)}{\partial \theta} - \frac{\partial Q(s,a,\theta)}{\partial \theta})) = 0.$$
    
By equallying to 0: 
    $$ (r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta))*(\gamma \frac{\partial \max_{a'}Q(s',a',\theta)}{\partial \theta} - \frac{\partial Q(s,a,\theta)}{\partial \theta}) = 0 \text{  almost sure}
 $$
 
 $\frac{\partial \max_{a'}Q(s',a',\theta)}{\partial \theta}$ is equal to 0, and $\frac{\partial Q(s,a,\theta)}{\partial \theta}$ is unlikely to be equal to 0
 
 Therefore, we have:
     $$ (r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)) = 0 \text{  almost sure} $$
 
 By definition, the Q founded as such is the $Q^*$ that we are looking for.

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [9]:
class Memory(object):
    def __init__(self, max_size=100):
        self.max_size = max_size
        self.memory = list()
    def is_full(self):
        return len(self.memory) >= self.max_size
    
    def remember(self, move):
        # move should a tuple (s_t, s_t+1, action, reward, game_over)
        if self.is_full():
            self.memory.pop(0)
        self.memory.append(move)

    def random_access(self):
        n = len(self.memory)
        random_index = np.random.randint(0, n)
        return self.memory[random_index]

***
The pipeline we will use for training is given below:

In [48]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [49]:
def format_input_model(state, action, n_action=4):
    x, y, z = np.shape(state)
#     input_states = np.zeros((x, y, z, n_action))
#     input_states[:, :, :, action] = state
    input_states = np.zeros((x, y, z))
    input_states[:, :, :] = state
    return input_states

class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2, discount = 0.99):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = discount
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        greedy_action, max_Q = self.argmax_Q(s)
        return greedy_action

    def reinforce(self, state, next_state, action, reward, game_over):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([state, next_state, action, reward, game_over])
        
        # 5 x 5 is the shape of a state
        # self.n_state = 2 is the dimension of the input of a state[i, j] 
        # giving (value in {0, -1, 0.5}, is_accessible in {0, -1})
        
        input_states = np.zeros((self.batch_size, 5, 5, self.n_state))
        # 4 is the number of actions possible, so target_Q isnt the max, but all Q possibles ? 
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            ######## FILL IN
            memory_state, memory_next_state, memory_action, memory_reward, memory_game_over = self.memory.random_access()
                
            input_states[i, :, :, :] = memory_state
            target_q[i, :] = self.Q(memory_state)
            
            if memory_game_over: # or memory_game_over ?
                target_q[i][memory_action] = memory_reward
            else:
                ######## FILL IN
                target_q[i][memory_action] = memory_reward + self.discount * self.max_Q(memory_next_state)
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)
        return l
    
    def max_Q(self, state):
        arg_max, max_Q = self.argmax_Q(state)
        return max_Q
    
    def argmax_Q(self, state):
        #available_actions = self.available_actions(state)
        Q_values = self.Q(state)[0] # self.Q() will return a 1 x 4 array of arrays.
        arg_max = np.argmax(Q_values)
        Q_values[arg_max]
        return (arg_max, Q_values[arg_max])
    
    def Q(self, state):
        return self.model.predict(np.array([state]))

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1, momentum=0.0, **kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        
        ####### FILL IN
        
        model = Sequential()
        model.add(Reshape((50, ), input_shape = (5, 5, 2)))
        model.add(Dense(30, activation="relu"))
        model.add(Dense(20, activation="relu"))
        model.add(Dense(4))
        
        model.compile(SGD(lr=lr, decay=1e-4, momentum=momentum), "mse")
        self.model = model
        

In [57]:
size = 13
T=200
temperature=0.3
epochs_train=21 # set small when debugging
epochs_test=21 # set small when debugging

env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, momentum=0.9, epsilon = 0.1, memory_size=2000, batch_size = 32, discount=0.9)
train(agent, env, epochs_train, prefix='fc_train')

Epoch 000/021 | Loss 0.0003 | Win/lose count 1.5/5.0 (-3.5)
Epoch 001/021 | Loss 0.0017 | Win/lose count 2.0/7.0 (-5.0)
Epoch 002/021 | Loss 0.0072 | Win/lose count 8.5/8.0 (0.5)
Epoch 003/021 | Loss 0.0033 | Win/lose count 2.5/0 (2.5)
Epoch 004/021 | Loss 0.0059 | Win/lose count 3.5/2.0 (1.5)
Epoch 005/021 | Loss 0.0020 | Win/lose count 7.0/1.0 (6.0)
Epoch 006/021 | Loss 0.0092 | Win/lose count 3.5/4.0 (-0.5)
Epoch 007/021 | Loss 0.0019 | Win/lose count 5.5/5.0 (0.5)
Epoch 008/021 | Loss 0.0037 | Win/lose count 4.0/2.0 (2.0)
Epoch 009/021 | Loss 0.0045 | Win/lose count 10.5/3.0 (7.5)
Epoch 010/021 | Loss 0.0034 | Win/lose count 6.0/2.0 (4.0)
Epoch 011/021 | Loss 0.0080 | Win/lose count 7.0/9.0 (-2.0)
Epoch 012/021 | Loss 0.0106 | Win/lose count 14.0/6.0 (8.0)
Epoch 013/021 | Loss 0.0049 | Win/lose count 7.0/6.0 (1.0)
Epoch 014/021 | Loss 0.0056 | Win/lose count 8.0/5.0 (3.0)
Epoch 015/021 | Loss 0.0080 | Win/lose count 12.0/3.0 (9.0)
Epoch 016/021 | Loss 0.0095 | Win/lose count 14.5/4

In [58]:
HTML(display_videos('fc_train20.mp4'))

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [55]:
class DQN_CNN(DQN):
    def __init__(self, *args, lr=0.1, momentum=0.0, **kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        ###### FILL IN
        model = Sequential()
        model.add(Conv2D(64, kernel_size = 3, activation="relu", input_shape = (5, 5, self.n_state)))
        model.add(Conv2D(32, kernel_size = 3, activation = "relu"))
        #model.add(MaxPooling2D((2,2)))
        model.add(Reshape((32, )))
        model.add(Dense(4))
        
        model.compile(SGD(lr=lr, decay=1e-4, momentum=momentum), "mse")
        self.model = model

In [67]:
cnn_epochs_train = 101
cnn_lr = 0.3

env = Environment(grid_size=size, max_time=T, temperature=0.3)
cnn_agent = DQN_CNN(size, lr=cnn_lr, momentum=0.9, epsilon = 0.1, memory_size=2000, batch_size = 32)
cnn_agent.model.summary()
train(cnn_agent, env, cnn_epochs_train, prefix='cnn_train')

Model: "sequential_25"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_38 (Conv2D)           (None, 3, 3, 64)          1216      
_________________________________________________________________
conv2d_39 (Conv2D)           (None, 1, 1, 32)          18464     
_________________________________________________________________
reshape_25 (Reshape)         (None, 32)                0         
_________________________________________________________________
dense_37 (Dense)             (None, 4)                 132       
Total params: 19,812
Trainable params: 19,812
Non-trainable params: 0
_________________________________________________________________


KeyboardInterrupt: 

In [62]:
HTML(display_videos('cnn_train20.mp4'))

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [68]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')

epochs_test = 21
print('Test of the CNN') # 12.238095238095237
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC') #9.738095238095237
test(agent_fc,env,epochs_test,prefix='fc_test') 

Test of the CNN
Win/lose count 19.5/3.0. Average score (16.5)
Win/lose count 19.0/4.0. Average score (15.75)
Win/lose count 16.0/0. Average score (15.833333333333334)
Win/lose count 17.5/3.0. Average score (15.5)
Win/lose count 13.5/3.0. Average score (14.5)
Win/lose count 22.5/3.0. Average score (15.333333333333334)
Win/lose count 15.0/3.0. Average score (14.857142857142858)
Win/lose count 18.5/4.0. Average score (14.8125)
Win/lose count 8.0/1.0. Average score (13.944444444444445)
Win/lose count 9.5/3.0. Average score (13.2)
Win/lose count 13.0/5.0. Average score (12.727272727272727)
Win/lose count 17.5/6.0. Average score (12.625)
Win/lose count 22.5/4.0. Average score (13.076923076923077)
Win/lose count 22.0/6.0. Average score (13.285714285714286)
Win/lose count 16.0/2.0. Average score (13.333333333333334)
Win/lose count 12.0/4.0. Average score (13.0)
Win/lose count 21.5/5.0. Average score (13.205882352941176)
Win/lose count 7.0/2.0. Average score (12.75)
Win/lose count 18.5/2.0. Ave

In [42]:
HTML(display_videos('cnn_test10.mp4'))

In [19]:
HTML(display_videos('fc_test20.mp4'))

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [34]:
def train_explore(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0
    list_win_scores = []
    list_lose_scores = []
    list_total_scores = []
    list_losses = []
    def set_epsilon_strategy(agent, state):
        total_nbr_of_cells = state.shape[0] * state.shape[1]
        malus_positions_cells = state[:, :, 2]
        
        nbr_of_malus_position = len(malus_positions_cells[malus_positions_cells > 0])
        
        ratio = nbr_of_malus_position / total_nbr_of_cells
        
        if ratio >= 0.75:
            agent.set_epsilon(0.7)
        elif ratio >= 0.5:
            agent.set_epsilon(0.5)
        elif ratio >= 0.25:
            agent.set_epsilon(0.3)
        else:
            agent.set_epsilon(0.1)

    
    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over, reward_without_malus_position = env.act(action, train = True)
            set_epsilon_strategy(agent, state)

            # Update the counters
            if reward_without_malus_position > 0:
                win = win + reward_without_malus_position
            if reward_without_malus_position < 0:
                lose = lose -reward_without_malus_position

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
                .format(e, epoch, loss, win, lose, win-lose))
        list_win_scores.append(win)
        list_lose_scores.append(lose)
        list_total_scores.append(score)
        list_losses.append(loss)
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
    return {"loss": list_losses, "score": list_total_scores, "win": list_win_scores, "lose": list_lose_scores}
#     pass
        
class EnvironmentExploring(Environment):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size))
        
        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))

    def act(self, action, train = False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        #self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
#         reward = self.board[self.x, self.y]
        reward = 0
        reward_without_malus_position = 0
        
        if train:
            reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1
        
        reward = reward + self.board[self.x, self.y]
        reward_without_malus_position = self.board[self.x, self.y]

        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time

        # 3 "feature" states instead of 2
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                        self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
            
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over, reward_without_malus_position

    def reset(self):
        """This function resets the game and returns the initial state"""
        
        # random between min=3 and max= grid_size - 3
        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]

        # Binom(1, temperature), size fois
        bonus = 0.5 * np.random.binomial(1, self.temperature, size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0 * np.random.binomial(1, self.temperature, size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))

        # Pour ne pas avoir les malus aux mêmes positions que les bonus
        malus[bonus>0]=0
        
        self.board = bonus + malus
        self.board[self.x,self.y] = 0

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        #self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        self.t = 0
        
        self.malus_position = np.zeros((self.grid_size, self.grid_size))
        
        # 3 "feature" states instead of 2
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                        self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        
        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state
    
## use those samples of code:
#In train explore:
# state, reward, game_over = env.act(action, train=True)

## In Environment exploring:
# You will have to change n_state to 3 because you will use one more layer!
#         reward = 0
#         if train:
#             reward = -self.malus_position[self.x, self.y]
#         self.malus_position[self.x, self.y] = 0.1

# reward = reward + self.board[self.x, self.y]
# # 3 "feature" states instead of 2
# state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
#                                 self.board.reshape(self.grid_size, self.grid_size,1),
#                         self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

In [44]:
# Training
cnn_epochs_train_explore = 21

env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=0.01, momentum=0.9, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
results = train_explore(agent, env, cnn_epochs_train_explore, prefix='cnn_train_explore')

HTML(display_videos('cnn_train_explore20.mp4'))

Epoch 000/021 | Loss 0.0069 | Win/lose count 4.5/9.0 (-4.5)
Epoch 001/021 | Loss 0.0066 | Win/lose count 5.0/9.0 (-4.0)
Epoch 002/021 | Loss 0.0051 | Win/lose count 9.5/6.0 (3.5)
Epoch 003/021 | Loss 0.0088 | Win/lose count 13.5/4.0 (9.5)
Epoch 004/021 | Loss 0.0044 | Win/lose count 8.0/0 (8.0)
Epoch 005/021 | Loss 0.0072 | Win/lose count 9.5/2.0 (7.5)
Epoch 006/021 | Loss 0.0054 | Win/lose count 9.0/4.0 (5.0)
Epoch 007/021 | Loss 0.0094 | Win/lose count 12.5/6.0 (6.5)
Epoch 008/021 | Loss 0.0129 | Win/lose count 14.5/5.0 (9.5)
Epoch 009/021 | Loss 0.0122 | Win/lose count 10.0/5.0 (5.0)
Epoch 010/021 | Loss 0.0053 | Win/lose count 15.0/5.0 (10.0)
Epoch 011/021 | Loss 0.0058 | Win/lose count 15.0/4.0 (11.0)
Epoch 012/021 | Loss 0.0527 | Win/lose count 22.5/3.0 (19.5)
Epoch 013/021 | Loss 0.0429 | Win/lose count 22.5/2.0 (20.5)
Epoch 014/021 | Loss 0.0037 | Win/lose count 10.5/1.0 (9.5)
Epoch 015/021 | Loss 0.0035 | Win/lose count 16.0/4.0 (12.0)
Epoch 016/021 | Loss 0.0053 | Win/lose co

In [45]:
HTML(display_videos('cnn_train_explore20.mp4'))

# plt.figure(figsize=(8, 6))
# plt.subplot(221)
# plt.plot(results['loss'])
# plt.ylabel('loss')
# plt.subplot(222)
# plt.plot(results['score'])
# plt.ylabel('score')
# plt.subplot(223)
# plt.plot(results['win'])
# plt.ylabel('win')
# plt.subplot(224)
# plt.plot(results['lose'])
# plt.ylabel('lose')


In [26]:
# Evaluation
epochs_test_explore = 21
test(agent,env,epochs_test_explore,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore20.mp4'))

Win/lose count 9.0/1.0. Average score (8.0)
Win/lose count 17.5/0. Average score (12.75)
Win/lose count 19.5/3.0. Average score (14.0)
Win/lose count 19.5/2.0. Average score (14.875)
Win/lose count 13.5/8.0. Average score (13.0)
Win/lose count 21.0/3.0. Average score (13.833333333333334)
Win/lose count 22.0/1.0. Average score (14.857142857142858)
Win/lose count 21.5/7.0. Average score (14.8125)
Win/lose count 15.0/3.0. Average score (14.5)
Win/lose count 18.5/5.0. Average score (14.4)
Win/lose count 22.0/3.0. Average score (14.818181818181818)
Win/lose count 24.5/5.0. Average score (15.208333333333334)
Win/lose count 12.5/2.0. Average score (14.846153846153847)
Win/lose count 23.5/3.0. Average score (15.25)
Win/lose count 15.5/0. Average score (15.266666666666667)
Win/lose count 14.5/2.0. Average score (15.09375)
Win/lose count 19.0/5.0. Average score (15.029411764705882)
Win/lose count 11.0/6.0. Average score (14.472222222222221)
Win/lose count 18.5/2.0. Average score (14.578947368421

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***